# Difference method + joint learning

In this model, we will use what we called the Difference method. We first calculate the difference between the experimental band gap and the PBE band gap, and we then evaluate the error on this difference. We have three targets in the dataframe: the experimental and PBE band gaps, and the difference between them. We will apply joint learning on these columns.

In [1]:
def setup_threading():
    import os
    os.environ['OPENBLAS_NUM_THREADS'] = '1'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["TF_NUM_INTRAOP_THREADS"] = "1"
    os.environ["TF_NUM_INTEROP_THREADS"] = "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
setup_threading()

In [3]:
from modnet.preprocessing import MODData
from modnet.models.vanilla import MODNetModel
from modnet.hyper_opt.fit_genetic import FitGenetic
import numpy as np
import os
import copy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [4]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.reindex([ids])
    data.df_targets = data.df_targets.reindex([ids])
    data.df_structure = data.df_structure.reindex([ids])
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    #data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

def MD_append(md,lmd):
    md = copy.deepcopy(md)
    for m in lmd:
        md.df_structure.append(m.df_structure)
        md.df_targets.append(m.df_targets)
        md.df_featurized.append(m.df_featurized)
    return md

In [3]:
#from random import randint
#def train_val_split(data):

#    """Splits arrays or matrices into random train and validation subsets.
#
#   Parameter:
#      data: 'MODData' data which need to be splitted.
#    """
    #i = randint(0,9)
    #md_train, md_val = MDKsplit(data, n_splits=5, random_state=1)[0]
    #y_train = md_train.df_targets
    #y_val = md_val.df_targets

    #return md_train, md_val, y_train, y_val

In [4]:
#train_val_split(md_joint)

In [5]:
md_exp = MODData.load('exp_gap_all_mpid')
md_exp.df_targets.columns = ['exp_gap','mp_id']
md_pbe = MODData.load('pbe_gap.zip')
md_pbe.df_targets.columns = ['gap']
md_joint = MODData.load('exp_pbe_joint')


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766

2021-05-05 11:51:24,381 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f54f42e8700> object, created with modnet version <=0.1.7
2021-05-05 11:51:31,026 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f54f42eeac0> object, created with modnet version <=0.1.7
2021-05-05 11:51:31,050 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f54f42da760> object, created with modnet version <=0.1.7


In [6]:
md_joint.df_targets

,exp_gap,pbe_gap
mp-12699,0.00,0.0000
mp-559459,3.40,2.1704
mp-21162,0.00,0.0000
mp-1306,0.00,0.0000
mp-15252,0.53,0.1049
...,...,...
mp-556541,1.80,1.5135
mp-2602,1.04,0.0000
mp-30366,0.00,0.0000
mp-3718,1.45,0.2418


In [7]:
md_joint.df_targets['difference'] = md_joint.df_targets['exp_gap'] - md_joint.df_targets['pbe_gap']

In [8]:
md_joint.df_targets

,exp_gap,pbe_gap,difference
mp-12699,0.00,0.0000,0.0000
mp-559459,3.40,2.1704,1.2296
mp-21162,0.00,0.0000,0.0000
mp-1306,0.00,0.0000,0.0000
mp-15252,0.53,0.1049,0.4251
...,...,...,...
mp-556541,1.80,1.5135,0.2865
mp-2602,1.04,0.0000,1.0400
mp-30366,0.00,0.0000,0.0000
mp-3718,1.45,0.2418,1.2082


In [9]:
import pymatgen.ext.matproj

In [10]:
from pymatgen.ext.matproj import MPRester
import pandas as pd

with MPRester("v454AlrxL5Zg0cLAXW0") as mpr:
    index = md_joint.df_structure.index
    array = np.empty(shape=md_joint.df_structure.shape, dtype=pymatgen.core.structure.Structure)
    array = np.ndarray.flatten(array)
    for i in range(len(md_joint.df_structure)):
        array[i] = mpr.get_structure_by_material_id(index[i])
    md_joint.df_structure = pd.DataFrame(data=array, columns=["structure"])
    md_joint.df_structure = md_joint.df_structure.set_index(index)

In [11]:
#md_joint.save('exp_pbe_joint')

2021-05-05 11:58:44,955 - modnet - INFO - Data successfully saved as exp_pbe_joint!


In [5]:
md_joint = MODData.load('exp_pbe_joint')


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766

2021-05-20 15:06:30,044 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f6a8c4d4d00> object, created with modnet version <=0.1.7


In [6]:
md_joint.df_structure

,structure
id,
mp-12699,"[[ 0.13616363 -2.60631371 1.50340165] Ca, [-1..."
mp-559459,"[[2.14049282 6.43464803 1.76199846] Cs, [-0.70..."
mp-21162,"[[0. 0. 0.] Sr, [0. 2.5259875 2.5259875..."
mp-1306,"[[3.259129 1.88166243 1.15971025] Th, [ 3.25..."
mp-15252,"[[0.95531657 1.042239 0.86229681] Cu, [2.803..."
...,...
mp-556541,"[[0. 6.49363948 5.82657217] In, [ 0. ..."
mp-2602,"[[3.023318 3.023318 3.023318] Nd, [0. 0. 0.] As]"
mp-30366,"[[4.51774558 1.37529211 4.32659015] Ca, [1.109..."


In [7]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(md_joint.df_targets['exp_gap'], md_joint.df_targets['pbe_gap'])

0.429783975697596

In [8]:
md_joint.df_targets['difference'].mad()

0.5494667780829975

In [9]:
import uncertainty_toolbox as uct

k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
errors = np.ones((int(len(md_joint.df_targets)/k),5))
maes = np.ones(5)
uncertainties = np.ones(5)
metricss = []
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_joint_exp-pbe_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
       
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    # difference prediction
    ga = FitGenetic(train)
    model = ga.run(size_pop=20, num_generations=10, n_jobs=20)
    
    pred, std = model.predict(test, return_unc=True)
    true = test.df_targets
    true = true.drop(columns=['exp_gap','pbe_gap'])
    metrics = uct.metrics.get_all_metrics(pred['difference'].values, std['difference'].values, true['difference'].values)
    error = pred-true
    error = error['difference'].drop(pred.index[((pred['difference']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    std = std['difference'].drop(std.index[((std['difference']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    uncertainty = np.abs(std).mean()
    print('MAE =')
    print(mae)
    print('uncertainty =')
    print(uncertainty)
    maes[i] = mae
    uncertainties[i] = uncertainty
    metricss.append(metrics)

2021-05-20 15:06:34,511 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-20 15:06:34,556 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-20 15:06:34,593 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-20 15:06:34,644 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-20 15:06:34,681 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-20 15:06:34,734 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-20 15:06:34,769 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-20 15:06:34,817 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-20 15:06:34,853 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-20 15:06:34,904 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-20 15:06:35,365 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f6a19a8a430> object, created with modnet version 0.1.9
2021-05-20 15

100%|██████████| 100/100 [13:37<00:00,  8.18s/it]

2021-05-20 15:20:16,412 - modnet - INFO - Loss per individual: ind 0: 0.867 	ind 1: 0.973 	ind 2: 0.921 	ind 3: 1.068 	ind 4: 0.879 	ind 5: 0.938 	ind 6: 0.903 	ind 7: 0.850 	ind 8: 0.951 	ind 9: 0.894 	ind 10: 0.826 	ind 11: 0.863 	ind 12: 0.964 	ind 13: 0.881 	ind 14: 0.864 	ind 15: 0.901 	ind 16: 0.885 	ind 17: 0.981 	ind 18: 0.859 	ind 19: 0.858 	


2021-05-20 15:20:19,512 - modnet - INFO - Generation number 1
2021-05-20 15:20:23,076 - modnet - INFO - Multiprocessing on 20 cores. Total of 32 cores available.


100%|██████████| 100/100 [11:02<00:00,  6.63s/it] 

...

 (3/n) Calculating adversarial group calibration metrics
  [1/2] for mean absolute calibration error
Measuring adversarial group calibration by spanning group size between 0.0 and 1.0, in 10 intervals


  0%|          | 0/10 [00:00<?, ?it/s]

  [2/2] for root mean squared calibration error
Measuring adversarial group calibration by spanning group size between 0.0 and 1.0, in 10 intervals


100%|██████████| 10/10 [00:02<00:00,  4.66it/s]

 (4/n) Calculating sharpness metrics
 (n/n) Calculating proper scoring rule metrics
**Finished Calculating All Metrics**


===================== Accuracy Metrics =====================
  MAE           0.186
  RMSE          0.410
  MDAE          0.010
  MARPD         155.946
  R2            0.496
  Correlation   0.719
=============== Average Calibration Metrics ================
  Root-mean-squared Calibration Error   0.084
  Mean-absolute Calibration Error       0.070
  Miscalibration Area                   0.070
========== Adversarial Group Calibration Metrics ===========
  Mean-absolute Adversarial Group Calibration Error
     Group Size: 0.11 -- Calibration Error: 0.112
     Group Size: 0.56 -- Calibration Error: 0.082
     Group Size: 1.00 -- Calibration Error: 0.070
  Root-mean-squared Adversarial Group Calibration Error
     Group Size: 0.11 -- Calibration Error: 0.139
     Group Size: 0.56 -- Calibration Error: 0.100
     Group Size: 1.00 -- Calibration Error: 0.084
==============

In [10]:
maes

array([0.33222279, 0.28476676, 0.31888674, 0.28262384, 0.18559184])

In [11]:
maes.mean()

0.2808183948155837

In [12]:
uncertainties

array([0.12359382, 0.14253537, 0.15704803, 0.14324321, 0.11988797])

In [13]:
uncertainties.mean()

0.13726168125867844

In [14]:
metricss

[{'accuracy': {'mae': 0.33222279051059017,
   'rmse': 0.6776748256732111,
   'mdae': 0.0662896976470948,
   'marpd': 146.9026570534816,
   'r2': 0.39913545425355534,
   'corr': 0.6472028851931801},
  'avg_calibration': {'rms_cal': 0.11160391760585173,
   'ma_cal': 0.08823721081785602,
   'miscal_area': 0.08911120047164932},
  'adv_group_calibration': {'ma_adv_group_cal': {'group_sizes': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
           0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]),
    'adv_group_cali_mean': array([0.43464646, 0.16839208, 0.13844949, 0.12402578, 0.11737287,
           0.11417893, 0.10807097, 0.10096897, 0.09919762, 0.08823721]),
    'adv_group_cali_stderr': array([5.94006196e-02, 2.81425271e-02, 1.60758724e-02, 1.66081478e-02,
           8.17050025e-03, 1.13574325e-02, 6.94458359e-03, 5.16110525e-03,
           4.17516084e-03, 1.46284728e-17])},
   'rms_adv_group_cal': {'group_sizes': array([0.        , 0.11111111, 0.22222222,

#### Conclusion

Relative percentage change of the error compared to MODNet_exp_GA reference: 19%